In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

import zipfile
import os




In [ ]:
from google.colab import files

uploaded = files.upload()


Saving archive (4).zip to archive (4).zip


In [ ]:
import zipfile

with zipfile.ZipFile("archive (4).zip", 'r') as zip_ref:
    zip_ref.extractall("rain_data")

df = pd.read_csv("rain_data/weatherAUS.csv")
df.head()


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [ ]:
df = pd.read_csv("rain_data/weatherAUS.csv")

# Drop columns with too many missing values
df.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1, inplace=True)

# Drop rows with missing target
df.dropna(subset=['RainTomorrow'], inplace=True)

# Fill remaining missing numeric values
df.fillna(df.mean(numeric_only=True), inplace=True)

# Label encode all object (string) columns
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col].astype(str))

# Separate features and targets
X = df.drop(['RainTomorrow', 'Rainfall'], axis=1)  # input features
y_class = df['RainTomorrow']  # for classification
y_reg = df['Rainfall']        # for regression

# Normalize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# To store results
acc_scores = []
mae_scores = []


In [ ]:
for fold, (train_index, val_index) in enumerate(kf.split(X_scaled)):
    print(f"\n--- Fold {fold + 1} ---")

    X_train, X_val = X_scaled[train_index], X_scaled[val_index]
    y_train_class, y_val_class = y_class.iloc[train_index], y_class.iloc[val_index]
    y_train_reg, y_val_reg = y_reg.iloc[train_index], y_reg.iloc[val_index]

    # --- Classification Model ---
    model_class = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model_class.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    model_class.fit(X_train, y_train_class, epochs=20, batch_size=32, verbose=0)

    y_pred_class = model_class.predict(X_val).flatten()
    y_pred_class_binary = (y_pred_class > 0.5).astype(int)
    acc = accuracy_score(y_val_class, y_pred_class_binary)
    acc_scores.append(acc)

    # --- Regression Model ---
    model_reg = Sequential([
        Dense(128, activation='relu', input_shape=(X.shape[1],)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    model_reg.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
    model_reg.fit(X_train, y_train_reg, epochs=20, batch_size=32, verbose=0)

    y_pred_reg = model_reg.predict(X_val).flatten()
    mae = mean_absolute_error(y_val_reg, y_pred_reg)
    mae_scores.append(mae)

    print(f"Fold {fold+1} - Classification Accuracy: {acc:.4f} | Regression MAE: {mae:.4f}")



--- Fold 1 ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 1 - Classification Accuracy: 0.8504 | Regression MAE: 1.8947

--- Fold 2 ---


...

889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Fold 4 - Classification Accuracy: 0.8540 | Regression MAE: 1.8897

--- Fold 5 ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


889/889 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step  
Fold 5 - Classification Accuracy: 0.8507 | Regression MAE: 1.8242


In [ ]:
print(f"\nAverage Classification Accuracy: {np.mean(acc_scores):.4f}")
print(f"Average Regression MAE: {np.mean(mae_scores):.4f}")



Average Classification Accuracy: 0.8517
Average Regression MAE: 1.8484


In [ ]:
# Save models
model_class.save("rainfall_classification_model.h5")
model_reg.save("rainfall_regression_model.h5")

# Save the scaler too
import joblib
joblib.dump(scaler, "scaler.pkl")


['scaler.pkl']